import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def clean_text(text):

    text = " " + text + " "
    text = text.replace("-"," ")
    text = text.replace("\n"," ")
    text = text.replace('\n[SECTION]\n', '<stop>')
    text = text.replace('[SECTION]', '<stop>')
    text = text.replace('\r', '')
    text = text.replace("[^a-zA-Z]+", " ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    text = text.lower() # lower
    text = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text = text.replace("'s", "")

    return text

txt = "h hahahaha u r funny's"
clean_text(txt)

# Sentiment Analysis Using Supervised Learning Approach

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm, tree
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder

### Re-share the files

In [23]:
# FROM JB

f = 'label/joobin_min.csv'
jb_df = pd.read_csv(f, index_col=0)
jb_df.reset_index(inplace=True, drop=True)
jb_df['Date'] = pd.to_datetime(jb_df['Date'])

jb_df = jb_df[(jb_df['Class'] == 1) | (jb_df['Class'] == -1)]
jb_df

,Date,Original,Corpus,Class
0,2003-06-25,members agreed current economic conditions pre...,member agree current economic condition prevai...,-1
2,2003-06-25,also considered aspects committees communicati...,also consider aspect committee communication m...,-1
5,2003-06-25,considerable uncertainty surrounded individual...,considerable uncertainty surround individual p...,-1
6,2003-06-25,information reviewed meeting suggested economy...,information review meeting suggest economy con...,-1
7,2003-06-25,consumer spending increased moderately housing...,consumer spending increase moderately housing ...,-1
...,...,...,...,...
1945,2005-01-11,economy seemed growing fairly strong pace desp...,economy seem grow fairly strong pace despite t...,-1
1946,2005-01-11,context members believed important continue re...,context member believe important continue remo...,1
1948,2005-01-11,participants noted forward looking elements st...,participant note forward looking element state...,1
1949,2005-01-11,meeting members concurred current statement st...,meeting member concur current statement struct...,1


In [24]:
# FROM GLENMOND

dir_name = 'label/glenmond'
files = os.listdir(dir_name)
merged = []

for f in files:
    filename, ext = os.path.splitext(f)
    
    if ext == '.csv':
        read = pd.read_csv(dir_name + '/' + f, index_col=0)
        
        merged.append(read)

glen_df = pd.concat(merged)
glen_df.reset_index(inplace=True, drop=True)
glen_df['Date'] = pd.to_datetime(glen_df['Date'])
glen_df = glen_df[(glen_df['Class'] == 1) | (glen_df['Class'] == -1)]
glen_df

,Date,Original,Corpus,Class
2,2017-03-05,staff review economic situation information re...,staff review economic situation information re...,-1
3,2017-03-05,month change overall consumer prices close co...,month change overall consumer price close comm...,-1
5,2017-03-05,total nonfarm payroll employment rose march ga...,total nonfarm payroll employment rise march ga...,1
6,2017-03-05,unemployment rate decreased,unemployment rate decrease,1
17,2017-03-05,recent readings key factors influence consumer...,recent reading key factor influence consumer s...,1
...,...,...,...,...
1639,2017-06-14,labor market conditions continued strengthen r...,labor market condition continued strengthen re...,1
1640,2017-06-14,unemployment rate fell,unemployment rate fell,1
1696,2017-06-14,information received since federal open market...,information receive since federal open market ...,1
1697,2017-06-14,job gains moderated solid average since beginn...,job gain moderate solid average since begin ye...,1


In [25]:
# FROM JAY

dir_name = 'label/Jay/minutes'
files = os.listdir(dir_name)
merged = []

# for jay
for f in files:
    filename, ext = os.path.splitext(f)
    
    if ext == '.csv':
        read = pd.read_csv(dir_name + '/' + f, index_col=0)
        
        merged.append(read)

j_df = pd.concat(merged)
j_df.reset_index(inplace=True, drop=True)
j_df['Date'] = pd.to_datetime(j_df['Date'])


f = 'label/jay_statements.csv'
f2 = pd.read_csv(f, index_col=0)
f2.reset_index(inplace=True, drop=True)
f2['Date'] = pd.to_datetime(f2['Date'])

temp = [f2, j_df]
jay_df = pd.concat(temp)
jay_df.reset_index(inplace=True, drop=True)
jay_df = jay_df[(jay_df['Class'] == 1) | (jay_df['Class'] == -1)]
jay_df

,Date,Original,Corpus,Class
7,2017-03-05,committee views slowing growth first quarter l...,committee view slow growth first quarter likel...,1
15,2017-03-05,committee expects economic conditions evolve m...,committee expect economic condition evolve man...,1
17,2020-12-16,federal reserve committed using full range too...,federal reserve commit use full range tool sup...,-1
24,2020-12-16,committee seeks achieve maximum employment inf...,committee seek achieve maximum employment infl...,-1
26,2020-12-16,committee decided keep target range federal fu...,committee decide keep target range federal fun...,-1
...,...,...,...,...
3109,2000-06-28,increasing though still tentative indications ...,increase though still tentative indication slo...,-1
3112,2000-06-28,members generally saw little risk deferring po...,member generally saw little risk defer policy ...,1
3114,2000-06-28,discussion balance of risks sentence press sta...,discussion balance of risks sentence press sta...,1
3116,2000-06-28,conclusion discussion committee voted authoriz...,conclusion discussion committee vote authorize...,1


In [26]:
# FOR ERICA

f = 'label/erica.csv'
erica_df = pd.read_csv(f, index_col=0)
erica_df.reset_index(inplace=True)
erica_df['Date'] = pd.to_datetime(erica_df['Date'])
erica_df = erica_df[(erica_df['Class'] == 1) | (erica_df['Class'] == -1)]
erica_df

,Date,Original,Corpus,Class
5,2011-01-26,among factors cited could affect level structu...,among factor cite could affect level structura...,1
6,2011-01-26,research reviewed suggested structural unemplo...,research review suggest structural unemploymen...,1
9,2011-01-26,participants stressed certain determinants une...,participant stress certain determinant unemplo...,1
11,2011-01-26,staff review economic situation information re...,staff review economic situation information re...,-1
12,2011-01-26,consumer spending rose strongly late last year...,consumer spending rise strongly late last year...,1
...,...,...,...,...
779,2011-02-11,meanwhile employment state local government se...,meanwhile employment state local government se...,-1
781,2011-02-11,percent september long duration unemployment ...,percent september long duration unemployment s...,-1
782,2011-02-11,initial claims unemployment insurance edged si...,initial claim unemployment insurance edge sinc...,-1
783,2011-02-11,however real disposable income declined third ...,however real disposable income decline third q...,-1


In [27]:
# FOR JAMAIS

f = 'label/jamais.csv'
jamais_df = pd.read_csv(f, index_col=0)
jamais_df.reset_index(inplace=True)
jamais_df['Date'] = pd.to_datetime(jamais_df['Date'])
jamais_df = jamais_df[(jamais_df['Class'] == 1) | (jamais_df['Class'] == -1)]
jamais_df

,Date,Original,Corpus,Class
0,2006-10-05,information reviewed meeting suggested economi...,information review meeting suggest economic ac...,1
1,2006-10-05,consumer spending posted sizable increase driv...,consumer spending post sizable increase driven...,1
2,2006-10-05,later quarter however pace consumer spending m...,later quarter however pace consumer spending m...,1
3,2006-10-05,business investment spending strengthened firs...,business investment spending strengthen first ...,1
5,2006-10-05,overall consumer prices jumped march higher en...,overall consumer price jumped march high energ...,1
...,...,...,...,...
647,2006-06-29,slowdown attributed part higher energy prices ...,slowdown attribute part high energy price also...,-1
660,2006-06-29,many noted number factors temporarily boosting...,many note number factor temporarily boost infl...,1
662,2006-06-29,addition rising rents boosting cost shelter co...,addition rise rent boost cost shelter contribu...,1
666,2006-06-29,nevertheless several factors cited potentially...,nevertheless several factor cite potentially s...,1


In [51]:
# ALLOCATION

# jay + erica
for_jay = jb_df.iloc[:600, :]
for_jay.reset_index(inplace=True, drop=True)
for_erica = jb_df.iloc[600:, :]
for_erica.reset_index(inplace=True, drop=True)

# jamais + jb
for_jamais = jay_df.iloc[:600, :]
for_jamais.reset_index(inplace=True, drop=True)
for_jb = jay_df.iloc[600:, :]
for_jb.reset_index(inplace=True, drop=True)

f = [for_jb, glen_df]
for_jb_final = pd.concat(f)
for_jb_final.reset_index(inplace=True, drop=True)

# glen + erica
for_erica_2 = jamais_df.iloc[:73, :]
for_erica_2.reset_index(inplace=True, drop=True)
f = [for_erica, for_erica_2]
for_erica_final = pd.concat(f)
for_erica_final.reset_index(inplace=True, drop=True)

for_glen_2 = jamais_df.iloc[73:, :]
for_glen_2.reset_index(inplace=True, drop=True)
f = [erica_df, for_glen_2]
for_glen = pd.concat(f)
for_glen.reset_index(inplace=True, drop=True)

In [58]:
for_jay.to_csv('label/round2/jay.csv')

In [59]:
for_erica_final.to_csv('label/round2/erica.csv')

In [60]:
for_jamais.to_csv('label/round2/jamais.csv')

In [61]:
for_jb_final.to_csv('label/round2/jb.csv')

In [62]:
for_glen.to_csv('label/round2/glen.csv')

### Read data from statement & minutes

In [3]:
# delete later
# for joobin
dir_name = 'label/joobin'
files = os.listdir(dir_name)
merged = []

for f in files:
    filename, ext = os.path.splitext(f)
    
    if ext == '.csv':
        read = pd.read_csv(dir_name + '/' + f, index_col=0)
        
        merged.append(read)


mins_df = pd.concat(merged)
mins_df.reset_index(inplace=True, drop=True)

In [5]:
mins_df.to_csv('joobin_min.csv')

In [2]:
# TEMPORARY CURRENT METHOD

dir_name = 'label/Jay/statements'
files = os.listdir(dir_name)
merged = []

# for jay
for f in files:
    filename, ext = os.path.splitext(f)
    
    if ext == '.csv':
        read = pd.read_csv(dir_name + '/' + f, index_col=0)
        
        merged.append(read)

st_df = pd.concat(merged)
st_df.reset_index(inplace=True, drop=True)
st_df.to_csv('jay.csv')

In [3]:
# TEMPORARY CURRENT METHOD

# for glenmond
dir_name = 'label/glenmond'
files = os.listdir(dir_name)
merged = []

for f in files:
    filename, ext = os.path.splitext(f)
    
    if ext == '.csv':
        read = pd.read_csv(dir_name + '/' + f, index_col=0)
        
        merged.append(read)

# for joobin
dir_name = 'label/joobin'
files = os.listdir(dir_name)
merged = []

for f in files:
    filename, ext = os.path.splitext(f)
    
    if ext == '.csv':
        read = pd.read_csv(dir_name + '/' + f, index_col=0)
        
        merged.append(read)


mins_df = pd.concat(merged)
mins_df.reset_index(inplace=True, drop=True)

In [4]:
### TEMPORARY MANUAL OVERRIDE OF THE CLASSES

# f = 'jamais.csv'
# mins2_df = pd.read_csv(f, index_col=0)
# mins2_df.reset_index(inplace=True)
# mins2_df['Date'] = pd.to_datetime(mins2_df['Date'])

merged = []

f = 'label/erica.csv'
f1 = pd.read_csv(f, index_col=0)
f1.reset_index(inplace=True)
f1['Date'] = pd.to_datetime(f1['Date'])

f = 'label/jamais.csv'
f2 = pd.read_csv(f, index_col=0)
f2.reset_index(inplace=True)
f2['Date'] = pd.to_datetime(f2['Date'])

merged = [f1, f2, mins_df]
mins_df = pd.concat(merged)
mins_df.reset_index(inplace=True, drop=True)

# Filter out Class 0: aka Neutral

mins_df = mins_df[mins_df['Class'] != 0]
st_df = st_df[st_df['Class'] != 0]
df = pd.concat([st_df, mins_df])
df.reset_index(inplace=True, drop=True)